In [44]:
import os
from dotenv import load_dotenv
load_dotenv()
import openai
import json
import ast
import time

In [3]:
# Set openai.api_key to the OPENAI environment variable
openai.api_key = os.environ["api_key"]

# Extract Education Background from Biography

- system_msg and prompt to instruct LLM to return information in dictionary format

In [50]:
def extract_education_background(text):
    system_msg = "I am an AI assistant designed to help you extract educational background information from a given text. \
    I understand that the text contains a biography and I am capable of identifying and extracting details about the person's education, \
    including their Bachelor's, Master's, and PhD degrees. I also understand that after extracting the following educational information from the text\
    I have to dispaly it in the following format: {'Bachelor Degree': 'Institution', 'Master Degree': 'Institution', 'PhD': 'Institution'}. \
    If there is no information about a particular degree in the text, please return None for the corresponding key."

#     text = "Dr. Alexei Sourin was born in Moscow and received his M.Eng. and Ph.D. degrees in computer graphics from the Moscow Engineering Physics Institute, Russia (MEPhI) in \
#     1983 and 1988, respectively. From 1983 to 1993 he was a researcher at MEPhI where he worked on different scientific visualization and computer animation projects. Since 1993 he \
#     held faculty positions at Nanyang Technological University (NTU) in Singapore, except a period from 1999 to 2000 when he was Associate Professor at Moscow Institute of Physics and\
#     Technology (MIPT). Currently he is a tenured Associate Professor with the School of Computer Science and Engineering at NTU. His research interests are in shape modeling,\
#     multi-modal interaction, music visualization, and automated visual inspection in manufacturing. Dr. Sourin published over 230 referred research papers and was invited to\
#     give talks to many scientific events. Dr. Sourin is also one of the pioneers of electronic education in Singapore. From 2012 to 2018, he was a Chair of the IFIP WG5.10\
#     Computer Graphics and Virtual Worlds. Dr. Sourin is a Senior Member of IEEE and a member of ACM SIGGRAPH. He is an associate editor of several international journals\
#     including The Visual Computer (Springer) and Transactions on Computational Science (Springer). He was on the program committees of over 120 international conferences.\
#     He is a coordinator of the International Conferences on Cyberworlds for which he was also General and Program Chair in many years."

    prompt = f"""
    Based on the biography below, tell me where did this person got his Bachelor's Degree, Master's Degree and PhD from. Return the results strictly in a dictionary format as such, {{"Bachelor Degree": "Institution", "Master Degree": "Institution", "PhD": "Institution"}}. If there is no information or if information is ambiguous about a particular degree in the text, please return "None" for the corresponding key. Also please make sure key and value is between double quote "string".

    Biography:

    {text}
    """
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": system_msg },
                    {"role": "user", "content": prompt}
            ])
    return response['choices'][0]['message']['content']

In [31]:
text =  "Jie Zhang is a Professor of the School of Computer Science and Engineering at NTU Singapore, leading the Computational Intelligence Group. He obtained Ph.D. in Cheriton School of Computer Science from University of Waterloo and was the recipient of Alumni Gold Medal in 2009. Then he joined NTU as an Assistant Professor and was promoted to Associate Professor in 2015. From 2017-2018, he was appointed as Tan Chin Tuan Exchange Fellowship, New York University. He was also an Adjunct Fellow, Singapore Institute of Manufacturing Technology (SIMTech), A*STAR, from 2020-2021. His papers have been published by top journals and conferences and won several best paper awards. Jie Zhang is also active in serving research communities."

response = extract_education_background(text=text)


In [ ]:
print(response)

In [51]:
raw_dir = './prof_raw_data/'
prefix = 'education'
gpt_dir = './gpt_data/'

dr_ntu_files = [os.path.join(raw_dir, f) for f in os.listdir(raw_dir) if os.path.isfile(os.path.join(raw_dir, f)) and 'dr_ntu' in f]

for file in dr_ntu_files[60:]:
    with open(file,'r') as f:
        prof_dict = json.load(f)
    biography = prof_dict['biography']
    name = prof_dict['full_name'].lower().replace(' ','_')
    education_detail_str = extract_education_background(biography)
    education_detail_dict = ast.literal_eval(education_detail_str.strip())
    for key,value in education_detail_dict.items():
        if value =="None":
            education_detail_dict[key] = None
    with open(f"{gpt_dir}{prefix}_{name}.json",'w') as f:
        json.dump(education_detail_dict,f)
    # only limited to 3 calls per minute
    time.sleep(25)


# Classify Type of Affiliation

- Two Classes
    - Academic Collaborators
    - External Collaborators

add an extra key value pair "is_school":"True" to that dictionary. If the value of the key "aff" is not an academic institution, 
    add an extra key value pair "is_school":"False" to the dictionary instead. Please only display the updated list of dictionary and nothing else.

    For example, if the information provided is as follows, [{{'name': 'Dacheng Tao','url': 'https://scholar.google.com/citations?hl=en&user=RwlJNLcAAAAJ','aff': 'The University of Sydney'}},{{'name': 'Yong Luo','url': 'https://scholar.google.com/citations?hl=en&user=zb1oVGIAAAAJ','aff': 'Wuhan University'}}], you should return me
    [{{'name': 'Dacheng Tao','url': 'https://scholar.google.com/citations?hl=en&user=RwlJNLcAAAAJ','aff': 'The University of Sydney','is_school':'True'}},{{'name': 'Yong Luo','url': 'https://scholar.google.com/citations?hl=en&user=zb1oVGIAAAAJ','aff': 'Wuhan University','is_school':'True'}}] as Wuhan University and The University of Sydney are both academic institution.


In [206]:
def classify_collaborators_affiliation(info):
    system_msg = "I am an AI assistant designed to help you identify whether a specified named entity is affiliated to educational institutions or not."



    prompt = f"""
    The Collaborator Information is a list of dictionary. First initialise a new empty list and for each dictionary in the collaboration information, if the corresponding value of the key "aff" is an academic institution, append "True" to newly initialised list, otherwise append "False". If the corresponding value of 'aff' key is and empty string, append "False". The length of the newly initialised list should match the number of dictionary in the collaboration information. Please only display the list of labels as the output.

    Collaborator Information:

    {info}
    """
    print(info)
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": system_msg },
                    {"role": "user", "content": prompt}
            ],
            temperature=0.0)
    return response['choices'][0]['message']['content']

In [193]:
goog_sch_files = [os.path.join(raw_dir, f) for f in os.listdir(raw_dir) if os.path.isfile(os.path.join(raw_dir, f)) and 'goog_sch' in f]


In [194]:
with open(goog_sch_files[0],'r')as f:
    author = json.load(f)

In [58]:
for file in goog_sch_files:
    with open(file,'r')as f:
        author_dict = json.load(f)
    updated_list = []
    # pass in in chunks of 10
    co_authors = author_dict['co_authors_url']
    if len(co_authors)<=10:
        response = classify_collaborators_affiliation(co_authors)
        ast.literal_eval(response.removeprefix('Updated Collaborator Information:\n\n').replace('\n',''))
        updated_list.extend()

    
    

In [195]:
len(author['co_authors_url'])

59

In [215]:
author['co_authors_url'][23]

{'name': 'Ping Wang',
 'url': 'https://scholar.google.com/citations?hl=en&user=3sIHxrcAAAAJ',
 'aff': 'York University, IEEE Fellow, York Research Chair, IEEE VTS Distinguished Lecturer'}

In [207]:
response = classify_collaborators_affiliation(author['co_authors_url'])

[{'name': 'Dacheng Tao', 'url': 'https://scholar.google.com/citations?hl=en&user=RwlJNLcAAAAJ', 'aff': 'The University of Sydney'}, {'name': 'Yong Luo', 'url': 'https://scholar.google.com/citations?hl=en&user=zb1oVGIAAAAJ', 'aff': 'Wuhan University'}, {'name': 'Haiyong Xie', 'url': 'https://scholar.google.com/citations?hl=en&user=irReiV0AAAAJ', 'aff': 'University of Science and Technology of China'}, {'name': 'Kyle Guan', 'url': 'https://scholar.google.com/citations?hl=en&user=3Vf3mqsAAAAJ', 'aff': ''}, {'name': 'Tat-Seng Chua', 'url': 'https://scholar.google.com/citations?hl=en&user=Z9DWCBEAAAAJ', 'aff': 'National University of Singapore'}, {'name': 'Dusit (Tao) Niyato', 'url': 'https://scholar.google.com/citations?hl=en&user=T8sVhLMAAAAJ', 'aff': 'Nanyang Technological University (NTU)'}, {'name': 'Dapeng Oliver Wu', 'url': 'https://scholar.google.com/citations?hl=en&user=sDRLr8gAAAAJ', 'aff': 'City University of Hong Kong'}, {'name': 'Chuan Heng Foh', 'url': 'https://scholar.google.

In [208]:
print(response)

Here is the list of labels indicating whether each collaborator is affiliated with an educational institution or not:

[True, True, True, False, True, True, True, True, True, True, True, False, True, True, True, True, False, True, True, False, False, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

In [163]:
import re

In [184]:
len(author['co_authors_url'])

59

In [181]:
ast.literal_eval(re.search(r"\[.*\]",response)[0])

TypeError: 'NoneType' object is not subscriptable

In [147]:
result[0]

"[{'name': 'Dacheng Tao', 'url': 'https://scholar.google.com/citations?hl=en&user=RwlJNLcAAAAJ', 'aff': 'The University of Sydney', 'is_school': 'True'}, {'name': 'Yong Luo', 'url': 'https://scholar.google.com/citations?hl=en&user=zb1oVGIAAAAJ', 'aff': 'Wuha... 'aff': 'Assistant Professor, Singapore Institute of Technology (SIT)', 'is_school': 'False'}, {'name': 'Di Wu', 'url': 'https://scholar.google.com/citations?hl=en&user=guhA4VoAAAAJ', 'aff': 'Professor of Computer Science, Sun Yat-Sen University', 'is_school': 'True'}]"

In [5]:
import streamlit as st
import streamlit.components.v1 as components


# Assuming data is in the following format
data = {
    "Author": ["Co-author_1", "Co-author_2"],
    "Co-author_1": ["Co-author_2", "Co-author_3"],
    "Co-author_2": ["Co-author_3"],
    "Co-author_3": []
}

def create_graph_for_author(author, data):
    net = Network(notebook=True,cdn_resources='in_line')
    edges = set()

    # Add nodes
    for a in data.keys():
        net.add_node(a)

    # Add edges for direct collaborations
    for co_author in data[author]:
        net.add_edge(author, co_author)
        edges.add((author, co_author))

    # Add connections between co-authors who have collaborated with each other before
    for co_author in data[author]:
        if co_author in data:
            for co_co_author in data[co_author]:
                if co_co_author != author and co_co_author in data[author] and (co_co_author, co_author) not in edges:
                    net.add_edge(co_author, co_co_author)
                    edges.add((co_author, co_co_author))
    return net

net = create_graph_for_author("Author", data)

# Save the graph as an HTML file and display it in the Jupyter notebook
net.show("graph.html")

# Display the HTML file in the Streamlit app
# with open('graph.html', 'r') as f:
#     components.html(f.read(), height=800)

graph.html


TypeError: 'IFrame' object is not subscriptable

In [ ]:
import streamlit as st
import pandas as pd

# Sample data
data = {
    'Name': ['Person 1', 'Person 2', 'Person 3', 'Person 4'],
    'Research Interest': ['Machine Learning, Computer Vision', 'Machine Learning', 'Computer Vision', 'NLP'],
}

df = pd.DataFrame(data)

# Streamlit app
st.title("Filter Table by Research Interests")

# Create a list of unique research interests from the DataFrame
research_interests = df['Research Interest'].str.split(', ').explode().unique()

# Multiselect filter for research interests
selected_interests = st.multiselect("Filter by Research Interests", research_interests)

# Filter the DataFrame based on the selected interests
filtered_df = df[df['Research Interest'].str.split(', ').apply(lambda x: any(item in selected_interests for item in x))]

# Display the filtered DataFrame
st.dataframe(filtered_df)
